In [1]:
import sys
sys.path.append('../')
import numpy as np
from GroupingAlgorithm import *
from HEEM_VQE_Functions import *
from utils import Label2Chain, get_backend_conectivity
from qiskit import IBMQ
from qiskit_nature.drivers import PyQuanteDriver
from qiskit_nature.problems.second_quantization.electronic import ElectronicStructureProblem
from qiskit_nature.mappers.second_quantization import ParityMapper, JordanWignerMapper, BravyiKitaevMapper
from qiskit_nature.converters.second_quantization.qubit_converter import QubitConverter
from qiskit_nature.transformers import FreezeCoreTransformer
from qiskit.opflow import converters
from qiskit.opflow.primitive_ops import Z2Symmetries

Obtaing the conectivity of a real quantum hardware.

In [2]:
IBMQ.load_account()
provider    = IBMQ.get_provider(hub='ibm-q-csic', group='internal', project='iff-csic') # put here your provider
backend_exp = provider.get_backend('ibmq_paris') # select a quantum computer
WC_exp      = get_backend_conectivity(backend_exp)

Groups for different molecules. The grouping strategies considered are: non grouping, TPB, Bell+TPB, 2Q+TPB and HEEM.

In [8]:
mc = 10

In [9]:
molecules = [ 'H .0 .0 .0; H .0 .0 0.761'  #0.88
             ,
             'Li 0.0 0.0 0.0; H 0.0 0.0 1.619'
             ,
            'H 0.0 0.0 -1.339; Be 0.0 0.0 0.0; H 0.0 0.0 1.339'
             ,
            'O 0.0 0.0 0.0; H 0.757 0.586 0.0; H -0.757 0.586 0.0'
#              ,
#              'H 0.0 0.0 0.0; F 0.0 0.0 0.995'
#              ,
#             'Cl 0.0 0.0 0.0; H 0.0 0.0 1.0' 
            ]


for molecule in molecules :
    
    print( 'molecule: '+molecule )

    driver = PyQuanteDriver(molecule)
    qmolecule = driver.run()
    problem = ElectronicStructureProblem(driver)

    # Generate the second-quantized operators
    second_q_ops = problem.second_q_ops()

    # Hamiltonian
    main_op = second_q_ops[0]

    # Setup the mapper and qubit converter
    mapper = JordanWignerMapper()

    converter = QubitConverter(mapper=mapper)

    # The fermionic operators are mapped to qubit operators
    num_particles = (problem.molecule_data_transformed.num_alpha,
                 problem.molecule_data_transformed.num_beta)
    qubit_op   = converter.convert(main_op, num_particles=num_particles)
    
    
    
    num_qubits = qubit_op.num_qubits
    WC=list(range(num_qubits))
    WC=list(permutations(list(range(num_qubits)),2))

    print('number of qubits:', num_qubits)

    paulis, coeff, labels = Label2Chain(qubit_op)
    print( 'number of Paulis:', len(coeff) )
    
    Color, Groups_tpb, Measurements_tpb = TPBgrouping(paulis)
    print( 'number of TPB groups:', len(Groups_tpb) )
        
    print('number of Bell groups:' )
    qubit_op_temp = grouping_shuffle( qubit_op,[4,3,1,2], WC,n_mc=mc )
    
    print('number of 2Q groups:' )
    qubit_op_temp = grouping_shuffle( qubit_op,[4,9,8,7,6,5,3,2,1], WC,n_mc=mc )
    
    print('number of HEEM groups:')
    qubit_op_temp = grouping_shuffle( qubit_op,[4,9,8,7,6,5,3,2,1], WC_exp,n_mc=mc )
    
    print('----------------------------------------')


molecule: H .0 .0 .0; H .0 .0 0.761
number of qubits: 4
number of Paulis: 15
number of TPB groups: 5
number of Bell groups:



The original order gives 2 groups
The best order found gives 2 groups
number of 2Q groups:



The original order gives 2 groups
The best order found gives 2 groups
number of HEEM groups:



The original order gives 2 groups
The best order found gives 2 groups
----------------------------------------
molecule: Li 0.0 0.0 0.0; H 0.0 0.0 1.619
number of qubits: 12
number of Paulis: 631
number of TPB groups: 136
number of Bell groups:



The original order gives 39 groups
The best order found gives 39 groups
number of 2Q groups:


...


The original order gives 41 groups
The best order found gives 41 groups
number of HEEM groups:



The original order gives 116 groups
The best order found gives 111 groups
----------------------------------------
molecule: O 0.0 0.0 0.0; H 0.757 0.586 0.0; H -0.757 0.586 0.0
number of qubits: 14
number of Paulis: 1086
number of TPB groups: 224
number of Bell groups:



The original order gives 58 groups
The best order found gives 54 groups
number of 2Q groups:



The original order gives 58 groups
The best order found gives 58 groups
number of HEEM groups:



The original order gives 153 groups
The best order found gives 149 groups
----------------------------------------


In [4]:
molecule = 'Li 0.0 0.0 0.0; H 0.0 0.0 1.619'

driver = PyQuanteDriver(molecule)
qmolecule = driver.run()

freezeCoreTransfomer = FreezeCoreTransformer(freeze_core=True, remove_orbitals=[3, 4])
problem = ElectronicStructureProblem(driver, q_molecule_transformers=[freezeCoreTransfomer])

# Generate the second-quantized operators
second_q_ops = problem.second_q_ops()

# Hamiltonian
main_op = second_q_ops[0]

# Setup the mapper and qubit converter
mapper = ParityMapper()

converter = QubitConverter(mapper=mapper, two_qubit_reduction=True)

# The fermionic operators are mapped to qubit operators
num_particles = (problem.molecule_data_transformed.num_alpha,
             problem.molecule_data_transformed.num_beta)
qubit_op   = converter.convert(main_op, num_particles=num_particles)

num_qubits = qubit_op.num_qubits
WC=list(range(num_qubits))
WC=list(permutations(list(range(num_qubits)),2))

print('number of qubits:', num_qubits)

paulis, coeff, labels = Label2Chain(qubit_op)
print( 'number of Paulis:', len(coeff) )

Color, Groups_tpb, Measurements_tpb = TPBgrouping(paulis)
print( 'number of TPB groups:', len(Groups_tpb) )

Groups_bell, Measurements_bell = grouping( paulis,[4,3,1,2],WC)
print('number of Bell groups:', len(Groups_bell) )

Groups_2Q1, Measurements_2Q1 = grouping( paulis,[4,9,8,7,6,5,3,2,1],WC)
print('number of 2Q groups v1:', len(Groups_2Q1) )

Groups_2Q2, Measurements_2Q2 = grouping( paulis,[7,6,5,4,9,8,3,2,1],WC)
print('number of 2Q groups v2:', len(Groups_2Q1) )

Groups_HEEM, Measurements_HEEM = grouping( paulis,[4,3,2,1],WC_exp)
print('number of HEEM groups :', len(Measurements_HEEM) )

print('----------------------------------------')


number of qubits: 4
number of Paulis: 100
number of TPB groups: 25
number of Bell groups: 15
number of 2Q groups v1: 11
number of 2Q groups v2: 11
number of HEEM groups : 9
----------------------------------------


In [5]:
Measurements_HEEM

[[[4, [2, 3]], [4, [0, 1]]],
 [[1, [2]], [3, [3]], [4, [0, 1]]],
 [[1, [3]], [3, [2]], [4, [0, 1]]],
 [[1, [0]], [3, [1]], [4, [2, 3]]],
 [[1, [1]], [3, [0]], [4, [2, 3]]],
 [[1, [2]], [1, [0]], [3, [3]], [3, [1]]],
 [[1, [2]], [1, [1]], [3, [3]], [3, [0]]],
 [[1, [3]], [1, [0]], [3, [2]], [3, [1]]],
 [[1, [3]], [1, [1]], [3, [2]], [3, [0]]]]

In [6]:
qubit_op_2 = grouping_shuffle( qubit_op,[4,3,2,1], WC, n_mc=50 )


The original order gives 15 groups
The best order found gives 11 groups


In [7]:
import qiskit.tools.jupyter
%qiskit_version_table